# Assignment 2, Question 2

In [3]:
library(sampler)
library(data.table)

In [4]:
group = c("A", "B", "C", "D")
sizes = c(123, 534, 321, 2842)
# estimate proportion
p = c(0.84, 0.71, 0.55, 0.36)
dat = data.table(type=group, N=sizes, p=p)

# Steps

1. We start with a proportional allocation, and estimate MOE for the whole sample and also only for group C. I used the `proportion engaged` as the best estimate of the proportion per strata to estimate MOE.
2. We try another allocation. In the solution below, I increase the sample size of stratum C (to 100) to reduce C's MOE
3. Finally, I compute the total MOE and C's MOE, and justify my decision
   
There is no unique/correct answer to this question. The goal of the exercise is to come out with a solution that increases the precision of group C without losing much on the overall precision

In [5]:
# proportional assignment
dat[, proportional := astrata(300, p=p, N=N)]

print(paste0('Proportional MOE stratification (total estimate):', serrst(dat$proportional, dat$N, p=dat$p)))
print(paste0('Proportional MOE group C: ', serr(dat[type=='C', proportional], N=321, p=dat[type=='C', p])))

# increase sample of group C to 100, so we can run more analysis on that group (there is no correct answer here)
dat[type!='C', fixed := astrata(300-100, p=p, N=N)]
dat[type=='C', fixed := 100]

print(paste0('Fixed MOE stratification (total estimate):', serrst(dat$fixed, dat$N, p=dat$p)))
print(paste0('Fixed MOE group C: ', serr(dat[type=='C', fixed], N=321, p=dat[type=='C', p])))

[1] "Proportional MOE stratification (total estimate):0.0516"
[1] "Proportional MOE group C: 0.1876"
[1] "Fixed MOE stratification (total estimate):0.0587"
[1] "Fixed MOE group C: 0.081"


In [6]:
# design weights

# proportional
dat[, proportional_weight := 1/(proportional/N)]

# fixed
dat[, fixed_weight := 1/(fixed/N)]

In [7]:
dat[, proportional_pop_estimate := proportional * proportional_weight]
dat[, fixed_pop_estimate := fixed * fixed_weight]

sum(dat$proportional_pop_estimate)
sum(dat$fixed_pop_estimate)
sum(sizes)

[1] 3820

[1] 3820

[1] 3820

In [8]:
dat

type,N,p,proportional,fixed,proportional_weight,fixed_weight,proportional_pop_estimate,fixed_pop_estimate
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
A,123,0.84,10,7,12.30000,17.57143,123,123
B,534,0.71,42,31,12.71429,17.22581,534,534
C,321,0.55,25,100,12.84000,3.21000,321,321
D,2842,0.36,223,162,12.74439,17.54321,2842,2842
